# Generate a Set of Modules

In this tutorial, we are going to show how we generate a module set from geometric primitives.

You can adapt the code and include your own modules or build some from mesh-files you have locally - feel free to explore!

In [ ]:
from datetime import datetime
from pathlib import Path

import numpy as np
import pinocchio as pin

from timor.Bodies import Body, Connector, Gender
from timor.Joints import Joint
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere
from timor.utilities.transformation import Transformation
from timor.utilities.spatial import rotX, rotY, rotZ

In [ ]:
# Let's define some principle parameters
density = 1200  # kg / m3 estimated overall material density
diameter = 80 / 1000  # the diameter of our links [m]
inner_diameter = 60 / 1000  # assume the links are hollow, with 5cm thick walls
sizes = (150 / 1000, 300 / 1000, 450 / 1000)  # we are going to build links in various lengths (sizes) [m]

date = datetime(2022, 9, 15)
ROT_X = Transformation.from_rotation(rotX(np.pi)[:3, :3])  # A shortcut we are going to use frequently

## Utility functions

Our plan is to generate a module set built mostly from simple cylinders. We use it for showcase and debugging purposes, so it doesn't have to be perfect.
Still, we want to be able to estimate dynamic properties of the modules such as massor inertia, so we are going to need some utility methods.

In [ ]:
def cylinder_mass(l: float, r: float, r_inner: float = 0) -> float:
    """Calculates the mass of a (hollow) cylinder"""
    mass = density * l * np.pi * r ** 2
    hollow_mass = density * l * np.pi * r_inner ** 2
    return mass - hollow_mass


def cylinder_inertia(l: float, r: float, r_inner: float = 0) -> pin.Inertia:
    """Calculates the inertia of a (hollow) cylinder, assuming a centered coordinate system"""
    mass = cylinder_mass(l, r, r_inner)
    lever = np.asarray([0, 0, 0])
    I = np.zeros((3, 3))
    I[0, 0] = (1 / 12) * mass * (3 * (r_inner ** 2 + r ** 2) + l ** 2)
    I[1, 1] = I[0, 0]
    I[2, 2] = .5 * mass * (r_inner ** 2 + r ** 2)

    return pin.Inertia(mass, lever, I)

## Let's get it started

The first modules we are building are simple, I-shaped links. In other words, these modules are hollow cylinders that can be connected to other modules.
When creating a module, make sure:

    - It has a unique ID within the module set
    - Each body in the module (here, it's only one body per module) has a unique ID
    - Each connector in the module has a unique ID
    - The connector orientations follow the Timor definition. (Their z-axes are pointing away from the module) --> This is why we have a rotation between body coordinate system and proximal connector

In [ ]:
def i_links() -> ModulesDB:
    """For every size, creates an I-shaped link (aka a cylinder) with two connectors."""
    links = ModulesDB()
    for size in sizes:
        module_header = ModuleHeader(ID='i_{}'.format(int(size * 100)),
                                     name='I shaped link {}-{}-{}'.format(diameter, diameter, int(size * 100)),
                                     date=date,
                                     author=['Jonathan Külz'],
                                     email=['jonathan.kuelz@tum.de'],
                                     affiliation=['Technical University of Munich']
                                     )
        connectors = (
            Connector(f'{int(diameter * 100)}-{i}',
                      ROT_X @ Transformation.from_translation([0, 0, size / 2]) if i == 0
                      else Transformation.from_translation([0, 0, size / 2]),
                      gender=Gender.f if i == 0 else Gender.m,
                      connector_type='default',
                      size=[diameter])
            for i in range(2)
        )
        geometry = Cylinder({'r': diameter / 2, 'z': size}, pose=Transformation.from_translation([0, 0, 0]))
        body = Body('i_{}'.format(int(size * 100)), collision=geometry, connectors=connectors,
                    inertia=cylinder_inertia(size, diameter / 2, inner_diameter / 2))
        links.add(AtomicModule(module_header, [body]))
    return links

We are going to call that function later - now let's go ahead, define L-shaped links: These links have one short side (with a fixed _offset_) and orthogonal to it, a longer cylinder of length _size_. We are estimating their inertia by the cylinders only and ignore the quarter-sphere shaped connection where they would be welded together if they were built from hardware cylinders.

In [ ]:
def l_links() -> ModulesDB:
    """For every size, creates an I-shaped link (aka two orthogonal cylinders) with two connectors."""
    links = ModulesDB()
    offset = 100 / 1000
    for size in sizes:
        module_header = ModuleHeader(ID='l_{}'.format(int(size * 100)),
                                     name='L shaped link {}-{}-{}'.format(offset, diameter, int(size * 100)),
                                     date=date,
                                     author=['Jonathan Külz'],
                                     email=['jonathan.kuelz@tum.de'],
                                     affiliation=['Technical University of Munich']
                                     )

        body2con1 = ROT_X
        body2cyl1 = Transformation.from_translation([0, 0, offset / 2])  # We define the first cylinder reference frame aligned with the symmetry axes of the cylinder
        body2con2 = Transformation.from_translation([0, 0, offset]) \
                    @ Transformation.from_rotation(rotY(np.pi / 2)[:3, :3]) \
                    @ Transformation.from_translation([0, 0, size])  # The distal connector is placed at the end of the second cylinder...
        body2cyl2 = body2con2 @ Transformation.from_translation([0, 0, -size / 2])  #...while the second cylinder itself has a reference frame in it's symmetry axes

        connectors = (
            Connector(f'{int(diameter * 100)}-{i}',
                      body2con1 if i == 0 else body2con2,
                      gender=Gender.f if i == 0 else Gender.m,
                      connector_type='default',
                      size=[diameter])
            for i in range(2)
        )

        cyl1 = Cylinder({'r': diameter / 2, 'z': offset}, pose=Transformation.from_translation([0, 0, offset / 2]))
        cyl2 = Cylinder({'r': diameter / 2, 'z': size}, pose=body2con2 @ Transformation.from_translation([0, 0, - size / 2]))
        elbow = Sphere({'r': diameter / 2}, pose=Transformation.from_translation([0, 0, offset]))  # This "elbow" connects the cylinders - as we ignore the inertia, we can define it as a full sphere
        geometry = ComposedGeometry((cyl1, cyl2, elbow))  # We combine all bodies to one common geometry

        I_c1 = cylinder_inertia(offset, diameter / 2, inner_diameter / 2)
        I_c2 = cylinder_inertia(size, diameter / 2, inner_diameter / 2)
        I = I_c1.se3Action(pin.SE3(body2cyl1.homogeneous)) + I_c2.se3Action(pin.SE3(body2cyl2.homogeneous))  # Combine the inertias of the cylinders to one module inertia

        body = Body('l_{}'.format(int(100 * size)), collision=geometry, connectors=connectors, inertia=I)
        links.add(AtomicModule(module_header, [body]))
    return links

For the **base** and the **end effector** we are going with an easy simplification: The base will just be a box, the end-effector is abstracted as a small sphere, so we can visually identify it's position in visualizations.

In [ ]:
def base() -> ModulesDB:
    """Creates a base connector attached to a box."""
    l = .1
    geometry = Box({'x': l, 'y': l, 'z': l}, pose=Transformation.from_translation([0, 0, l / 2]))
    c_world = Connector('base', ROT_X, gender=Gender.f, connector_type='base', size=[diameter])
    c_robot = Connector('base2robot', gender=Gender.m, connector_type='default', size=[diameter],
                        body2connector=Transformation.from_translation([l / 2, 0, l / 2]) @ rotY(np.pi / 2) @ rotZ(np.pi))
    return ModulesDB({
        AtomicModule(ModuleHeader(ID='base', name='Base', author=['Jonathan Külz'], date=date,
                                  email=['jonathan.kuelz@tum.de'], affiliation=['Technical University of Munich']),
                     [Body('base', collision=geometry, connectors=[c_world, c_robot])])
    })


def eef() -> ModulesDB:
    """Creates a simplified end effector module."""
    geometry = Sphere({'r': diameter / 5}, pose=Transformation.from_translation([0, 0, diameter / 2]))
    c_robot = Connector('robot2eef', ROT_X, gender=Gender.f, connector_type='default', size=[diameter])
    c_world = Connector('end-effector', gender=Gender.m, connector_type='eef',
                        body2connector=Transformation.from_translation([0, 0, diameter / 2]))
    return ModulesDB({
        AtomicModule(ModuleHeader(ID='eef', name='Demo EEF', author=['Jonathan Külz'], date=date,
                                  email=['jonathan.kuelz@tum.de'], affiliation=['Technical University of Munich']),
                     [Body('EEF', collision=geometry, connectors=[c_robot, c_world])])
    })

## Last but not least: Build some Joints

We are going to define two joints:

    1. A linear joint which is abstracted by two cylinders of different radius moving relative to each other along their longitudal axis
    2. A prismatic/revolute joint, shaped like one of the L-shaped links but with a relative movement between the two cylinders

In [ ]:
def linear_joint() -> ModulesDB:
    """Creates an I-shaped prismatic joint"""
    part_length = 100 / 1000
    joint_limit = .6 * part_length
    proximal = Body('J1_proximal', collision=Cylinder({'r': diameter / 2, 'z': part_length}),
                    connectors=[Connector('J1_proximal',
                                          ROT_X @ Transformation.from_translation([0, 0, part_length / 2]),
                                          gender=Gender.f,
                                          connector_type='default',
                                          size=[diameter])],
                    inertia=cylinder_inertia(part_length, diameter / 2, inner_diameter / 2)
                    )
    distal = Body('J1_distal', collision=Cylinder({'r': 0.8 * diameter / 2, 'z': part_length + joint_limit}),
                  connectors=[Connector('J1_distal',
                                        Transformation.from_translation([0, 0, (part_length + joint_limit) / 2]),
                                        gender=Gender.m,
                                        connector_type='default',
                                        size=[diameter])],
                  inertia=cylinder_inertia(part_length * 2, .8 * diameter / 2, .8 * inner_diameter / 2)
                  )
    joint = Joint(
        joint_id='Prismatic',
        joint_type='prismatic',
        parent_body=proximal,
        child_body=distal,
        q_limits=[0, joint_limit],
        parent2joint=Transformation.from_translation([0, 0, part_length / 2]),  # Here we define where the movement axis of the joint is placed
        joint2child=Transformation.from_translation([0, 0, (part_length - joint_limit) / 2])  # Here we define where the movement axis of the joint is placed
    )

    module_header = ModuleHeader(ID='J1',
                                 name='Prismatic Joint',
                                 date=date,
                                 author=['Jonathan Külz'],
                                 email=['jonathan.kuelz@tum.de'],
                                 affiliation=['Technical University of Munich']
                                 )
    return ModulesDB({
        AtomicModule(module_header, [proximal, distal], [joint])
    })

In [ ]:
def revolute_joint() -> ModulesDB:
    """Creates an L-shaped joint"""
    length = 150 / 1000
    proximal_body = ComposedGeometry((Cylinder({'r': diameter / 2, 'z': length}),
                                      Sphere({'r': diameter / 2}, pose=Transformation.from_translation([0, 0, length / 2])))
                                     )
    proximal = Body('J2_proximal', collision=proximal_body,
                    connectors=[Connector('J2_proximal',
                                          ROT_X @ Transformation.from_translation([0, 0, length / 2]),
                                          gender=Gender.f,
                                          connector_type='default',
                                          size=[diameter])],
                    inertia=cylinder_inertia(length, diameter / 2, inner_diameter / 2)
                    )
    distal = Body('J2_distal', collision=Cylinder({'r': diameter / 2, 'z': 150 / 1000}),
                  connectors=[Connector('J2_distal',
                                        Transformation.from_translation([0, 0, length / 2]),
                                        gender=Gender.m,
                                        connector_type='default',
                                        size=[diameter])],
                  inertia=cylinder_inertia(length, diameter / 2, inner_diameter / 2)
                  )
    joint = Joint(
        joint_id='Revolute',
        joint_type='revolute',
        parent_body=proximal,
        child_body=distal,
        q_limits=np.array([-np.pi, np.pi]),
        torque_limit=1000,
        acceleration_limit=5,
        velocity_limit=10,
        parent2joint=Transformation.from_translation([0, 0, length / 2]) @ Transformation.from_rotation(rotY(np.pi / 2)[:3, :3]),
        joint2child=Transformation.from_translation([0, 0, length / 2])
    )

    module_header = ModuleHeader(ID='J2',
                                 name='Revolute Joint',
                                 date=date,
                                 author=['Jonathan Külz'],
                                 email=['jonathan.kuelz@tum.de'],
                                 affiliation=['Technical University of Munich']
                                 )
    return ModulesDB({
        AtomicModule(module_header, [proximal, distal], [joint])
    })

## It's done: Now let's see the modules

We are combining all modules defined to a common database.

Timor offers some visualization capabilities for debugging created databases. It shows the modules, but also all important reference frames defined - take a look, change some parameters and inspect how the visualized modules behave.

In [ ]:
db = i_links().union(l_links()).union(base()).union(eef()).union(linear_joint()).union(revolute_joint())
db.debug_visualization()